In [2]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)
import pandas as pd

In [3]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../www/data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,EGRESO
1,NUEVO BENEFICIARIO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


# Big Table with Summary

In [ ]:
query_summary = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [ ]:
final_table_to_query = executes_query_duckdb(query_summary, cae)

In [ ]:
final_table_to_query

,id,año_solicitud,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado,porcentaje_financiado
0,724.699,2013,F,Municipal,REGION METROPOLITANA,quintil 1 y 2,1,2,6,1,0,13579123.0,0.5
1,380.733,2013,M,Municipal,REGION METROPOLITANA,quintil 1 y 2,1,1,2,1,0,3806320.0,0.9
2,505.693,2012,M,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,2,1,0,3481687.0,0.9
3,21.641,2013,M,Particular subvencionado,REGION DEL MAULE,quintil 1 y 2,1,1,7,1,0,20271604.0,0.5
4,60.493,2013,F,Corporación municipal,REGION METROPOLITANA,quintil 5,1,1,2,1,0,3218553.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.240.230,2024,F,Particular Subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,2376000.0,0.8
1290989,1.200.946,2024,M,Particular Pagado,REGION DE LOS LAGOS,quintil 1 y 2,1,1,1,0,0,2846000.0,0.3
1290990,1.241.909,2024,M,Municipal DAEM,REGION DEL LIB. B. O'HIGGINS,quintil 5,1,1,1,0,0,6424646.0,1.0
1290991,1.215.941,2024,F,Particular Pagado,REGION METROPOLITANA,quintil 5,1,1,1,0,0,6343794.0,1.0


---

---

# Aggregated Total Borrowed

In [14]:
DOLAR = 950 # 1 dollar is $950 CLP

query_total_borrowed = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_licitacion as año_financiado,
    round(SUM(total_prestado_ajustado)/{DOLAR}, 0) as total_prestado
FROM cae
GROUP BY region, quintil, genero, año_financiado
"""

total_borrowed = executes_query_duckdb(query_total_borrowed, cae)

mapping = {'REGION METROPOLITANA': 13,
           'REGION DEL BIO-BIO': 8,
           'REGION DEL MAULE': 7,
           'REGION DE VALPARAISO': 5,
           'REGION DE LOS RIOS': 14,
           'REGION DE COQUIMBO': 4,
           "REGION DEL LIB. B. O'HIGGINS": 6,
           'REGION DE LOS LAGOS': 10,
           'REGION DE LA ARAUCANIA': 9,
           'REGION DE ATACAMA': 3,
           'REGION DE TARAPACA': 1,
           'REGION DE ARICA Y PARINACOTA': 15,
           'REGION DE ÑUBLE': 16,
           'REGION DE ANTOFAGASTA': 2,
           'REGION DE MAGALLANES Y LA ANTARTICA': 12,
           'REGION DE AYSEN DEL G. C. IBAÑEZ': 11}

#total_borrowed["region"] = total_borrowed["region"].map(mapping)

total_borrowed.to_json("../www/data/total_borrowed.json",
                        orient="records",
                        lines=False,
                        force_ascii=False,
                        indent=4)

p = total_borrowed.query("año_financiado == 2024")
p["total_prestado"].sum()/1000000

np.float64(294.59439)

---

---

# **Total CAES Granted**

In [ ]:
query_total_granted = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    COUNT(tipo_beneficiario) as prestamos_entregados
FROM cae
WHERE tipo_beneficiario = 'NUEVO BENEFICIARIO'
GROUP BY region, quintil, genero, año_financiado
"""

total_granted = executes_query_duckdb(query_total_granted, cae)
total_granted

# total_granted.to_json("../www/data/total_cae_granted.json",
#                         orient="records",
#                         lines=False,
#                         force_ascii=False,
#                         indent=4)

,region,quintil,genero,año_financiado,prestamos_entregados
0,REGION DE VALPARAISO,quintil 4,M,2019,510
1,REGION DE LOS LAGOS,quintil 5,F,2019,528
2,REGION DE VALPARAISO,quintil 5,M,2019,1326
3,REGION METROPOLITANA,quintil 4,M,2019,2187
4,REGION DE MAGALLANES Y LA ANTARTICA,quintil 4,F,2019,52
...,...,...,...,...,...
2299,REGION DE ATACAMA,quintil 3,F,2006,7
2300,REGION DE ARICA Y PARINACOTA,quintil 3,M,2020,53
2301,REGION DE MAGALLANES Y LA ANTARTICA,quintil 4,M,2006,14
2302,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2008,13


---

---

# **AVG LOAN**

In [ ]:
DOLAR = 950 # 1 dollar is $950 CLP

query_avg_loan = f"""
WITH money_per_loan as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        ROUND(SUM(total_prestado_ajustado)/{DOLAR},0) as total_prestado,
    FROM cae
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(total_prestado),0) as prestamo_promedio,
    COUNT(total_prestado) as prestamos_entregados

FROM money_per_loan
GROUP BY region, quintil, genero, año_entrega
"""

avg_loan = executes_query_duckdb(query_avg_loan, cae)

# avg_loan.to_json("../www/data/avg_loan.json",
#                          orient="records",
#                          lines=False,
#                          force_ascii=False,
#                          indent=4)

avg_loan

,region,quintil,genero,año_entrega,prestamo_promedio,prestamos_entregados
0,REGION METROPOLITANA,quintil 4,M,2011,12508.0,3846
1,REGION METROPOLITANA,quintil 1 y 2,M,2008,10915.0,4675
2,REGION DE VALPARAISO,quintil 3,M,2010,10949.0,766
3,REGION METROPOLITANA,quintil 1 y 2,F,2008,10293.0,6906
4,REGION DE LOS LAGOS,quintil 4,M,2009,12772.0,216
...,...,...,...,...,...,...
2356,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2022,5126.0,8
2357,REGION DE COQUIMBO,quintil 5,F,2008,8952.0,1
2358,REGION DEL MAULE,quintil 5,M,2009,13831.0,4
2359,REGION DE LA ARAUCANIA,quintil 5,F,2008,17003.0,2


---

---

# **FINANCED YEARS**

In [ ]:
query_financed_years = f"""
WITH prov_years_financed as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        COUNT(total_prestado_ajustado) as años_financiados
    FROM cae
    WHERE total_prestado_ajustado > 0
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(años_financiados),2) as años_financiados,
    COUNT(años_financiados) as prestamos_entregados

FROM prov_years_financed
GROUP BY region, quintil, genero, año_entrega
"""

financed_years = executes_query_duckdb(query_financed_years, cae)

# financed_years.to_json("../www/data/financed_years.json",
#                           orient="records",
#                           lines=False,
#                           force_ascii=False,
#                           indent=4)

financed_years

,region,quintil,genero,año_entrega,años_financiados,prestamos_entregados
0,REGION DE MAGALLANES Y LA ANTARTICA,quintil 1 y 2,M,2013,3.29,80
1,REGION METROPOLITANA,quintil 1 y 2,M,2015,3.15,7292
2,REGION METROPOLITANA,quintil 1 y 2,F,2011,3.93,14195
3,REGION METROPOLITANA,quintil 4,F,2015,3.64,3914
4,REGION METROPOLITANA,quintil 3,M,2013,3.81,4026
...,...,...,...,...,...,...
2356,REGION DE VALPARAISO,quintil 5,F,2009,6.00,3
2357,REGION DE TARAPACA,quintil 5,M,2006,5.38,8
2358,REGION DE ATACAMA,quintil 3,M,2006,4.00,1
2359,REGION DE LA ARAUCANIA,quintil 5,F,2010,4.00,2


---

---

# **Percentage Financed**

In [ ]:
query_percent_fin = f"""
WITH prov_percent_financed as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        AVG(porcentaje_financiado) as porcentaje_financiado
    FROM cae
    WHERE total_prestado_ajustado > 0
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(porcentaje_financiado),2) as porcentaje_financiado,
    COUNT(porcentaje_financiado) as prestamos_entregados

FROM prov_percent_financed
GROUP BY region, quintil, genero, año_entrega
"""

percent_fin = executes_query_duckdb(query_percent_fin, cae)

percent_fin.to_json("../www/data/percent_financed.json",
                           orient="records",
                           lines=False,
                           force_ascii=False,
                           indent=4)

percent_fin

,region,quintil,genero,año_entrega,porcentaje_financiado,prestamos_entregados
0,REGION DE VALPARAISO,quintil 4,M,2011,0.89,741
1,REGION METROPOLITANA,quintil 4,M,2011,0.93,3833
2,REGION DE VALPARAISO,quintil 1 y 2,F,2010,0.91,3877
3,REGION DE LOS LAGOS,quintil 1 y 2,M,2012,0.85,864
4,REGION DE LOS LAGOS,quintil 3,F,2014,0.82,391
...,...,...,...,...,...,...
2356,REGION DEL BIO-BIO,quintil 5,F,2010,0.90,1
2357,REGION DE ANTOFAGASTA,quintil 5,M,2010,1.00,1
2358,REGION DE LOS RIOS,quintil 5,F,2009,0.89,2
2359,REGION DE LOS LAGOS,quintil 5,F,2009,0.87,1


---

---

# **Path of each student**

In [ ]:
query_path = """
WITH student_career as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        /* (nombre_carrera || ' - ' || nombre_ies || ' - ' || tipo_ies) as id_carrera, */
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' then tipo_beneficiario END)
            as nuevo_beneficiario_años,
        COUNT(CASE WHEN tipo_beneficiario = 'BENEFICIARIO RENOVANTE' then tipo_beneficiario END)
            as renovante_años,
        COUNT(CASE WHEN tipo_beneficiario = 'SIN CREDITO POR NO MATRICULA O POR SUSPENSION' then tipo_beneficiario END)
            as suspendido_años,
        COUNT(CASE WHEN tipo_beneficiario = 'GRATUIDAD' then tipo_beneficiario END)
            as gratuidad_años,
        MIN(CASE WHEN tipo_beneficiario = 'EGRESO' then año_operacion END)
            as año_egreso,
        MIN(CASE WHEN tipo_beneficiario = 'DESERCION' then año_operacion END)
            as año_desercion,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados_cae,
        MIN(CASE WHEN cambio_ies_carrera = 'SI' then año_operacion END)
            as año_nueva_carrera


    FROM cae
    GROUP BY id,
            año_entrega,
            region,
            quintil,
            genero
            -- id_carrera
    ),

    ids_count as (
        SELECT id, COUNT(id) as total_records
        FROM student_career
        GROUP BY id),

    ids_clean as (
        SELECT id
        FROM ids_count
        WHERE total_records < 3
        ),
    final_paths as (
        SELECT sc.*
        FROM student_career sc
        INNER JOIN ids_clean ids
        ON sc.id = ids.id
        )

    
    SELECT
        año_entrega,
        region,
        quintil,
        genero,
        COUNT(DISTINCT(id)) as prestamos_entregados,
        COUNT(DISTINCT(CASE WHEN año_egreso > 0 AND año_nueva_carrera IS NULL AND suspendido_años = 0 then id END)) as egresados_directo,
        COUNT(DISTINCT(CASE WHEN año_egreso > 0 AND año_nueva_carrera IS NULL AND suspendido_años > 0 then id END)) as egresados_congelado,
        
        COUNT(DISTINCT(CASE WHEN año_desercion > 0 AND año_nueva_carrera IS NULL AND suspendido_años = 1 then id END)) as desertor_directo,
        COUNT(DISTINCT(CASE WHEN año_desercion > 0 AND año_nueva_carrera IS NULL AND suspendido_años > 1 then id END)) as desertor_congelado,
        
        COUNT(DISTINCT(CASE WHEN año_egreso > 0 AND año_nueva_carrera > 0 then id END)) as egresados_cambio_carrera,
        COUNT(DISTINCT(CASE WHEN año_desercion > 0 AND año_nueva_carrera > 0 then id END)) as desertor_cambio_carrera

    FROM final_paths
    WHERE año_egreso > 0 or año_desercion > 0
    GROUP BY
        año_entrega,
        region,
        quintil,
        genero
"""
path_students = executes_query_duckdb(query_path, cae)

# path_students.to_json("../www/data/path_student.json",
#                          orient="records",
#                          lines=False,
#                          force_ascii=False,
#                          indent=4)

path_students

,año_entrega,region,quintil,genero,prestamos_entregados,egresados_directo,egresados_congelado,desertor_directo,desertor_congelado,egresados_cambio_carrera,desertor_cambio_carrera
0,2011,REGION DE VALPARAISO,quintil 1 y 2,M,2776,1094,229,457,113,521,362
1,2006,REGION DEL LIB. B. O'HIGGINS,quintil 1 y 2,F,167,70,51,22,4,15,5
2,2009,REGION DEL BIO-BIO,quintil 1 y 2,M,2993,1525,274,418,136,423,217
3,2014,REGION METROPOLITANA,quintil 1 y 2,M,6632,3162,664,1181,351,814,460
4,2011,REGION DEL BIO-BIO,quintil 1 y 2,M,3819,1658,304,594,138,659,466
...,...,...,...,...,...,...,...,...,...,...,...
2221,2008,REGION DEL MAULE,quintil 5,M,1,0,0,0,0,1,0
2222,2021,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,6,3,1,2,0,0,0
2223,2020,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 4,F,5,3,0,1,0,1,0
2224,2010,REGION DE VALPARAISO,quintil 5,M,1,0,0,0,0,1,0


---

---

# **Total Enrolled**

In [28]:
total_enrolled = pd.read_json("../www/data/total_enrolled.json")

students_studying = """
Select
    año_operacion as año,
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    COUNT(DISTINCT(id_2023)) as total_students_cae
FROM cae
WHERE
    tipo_beneficiario = 'NUEVO BENEFICIARIO' or tipo_beneficiario = 'BENEFICIARIO RENOVANTE' OR tipo_beneficiario = 'GRATUIDAD'
GROUP BY año, region, quintil, genero
"""

studying_df = executes_query_duckdb(students_studying, cae)

merged_df = total_enrolled.merge(studying_df, on="año", how="inner")

merged_df["share_cae"] = merged_df["total_students_cae"] / merged_df["total matrícula"]
merged_df["share_cae"] = round(merged_df["share_cae"]*100,4)

merged_df.to_json("../www/data/students_studying.json",
                          orient="records",
                          lines=False,
                          force_ascii=False,
                          indent=4)

merged_df

,año,total matrícula,region,quintil,genero,total_students_cae,share_cae
0,2007,748344,REGION DEL MAULE,quintil 1 y 2,F,1274,0.1702
1,2007,748344,REGION DE VALPARAISO,quintil 3,M,495,0.0661
2,2007,748344,REGION DE LA ARAUCANIA,quintil 1 y 2,F,1017,0.1359
3,2007,748344,REGION DE LOS LAGOS,quintil 3,M,123,0.0164
4,2007,748344,REGION DE ARICA Y PARINACOTA,quintil 4,F,45,0.0060
...,...,...,...,...,...,...,...
2299,2024,1277611,REGION DE ÑUBLE,quintil 3,F,481,0.0376
2300,2024,1277611,REGION DE TARAPACA,quintil 3,F,176,0.0138
2301,2024,1277611,REGION DE LA ARAUCANIA,quintil 3,M,407,0.0319
2302,2024,1277611,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 1 y 2,M,72,0.0056
